In [ ]:
import math
import sys

import transforms
import vectors

Import `draw_model` function from `draw_model.py`, and then add workarounds to a few issues in this function.

In [ ]:
from draw_model import draw_model

# The draw_model function ends by calling quit(), which doesn't exist in jupyter notebook.
# Work around it by catching the resulting exception.
_draw_model = draw_model
def draw_model(*args, **kwargs):
    try:
        _draw_model(*args, **kwargs)
    except NameError as e:
        if "'quit'" not in e.args[0]:
            raise

# When the get_matrix parameter is specified, draw_model tries to call multiply_matrix_vector,
# which is not defined anywhere. Adding it here. The original intention should be using matrix
# multiplication to transform the vertex, but since the rest of the chapter uses transform
# function, I am using that here as well. (Transform by matrix will be covered in Chapter 5.)
sys.modules['draw_model'].multiply_matrix_vector = lambda f, x: f(x)

A modified version of the code from `teapot.py`. In addtion to some changes in the implementation details, the most significant difference is the original code applies some transformations to the vertices as they are loaded, while the new code makes no such transformations.

In [ ]:
def load_vertices(vertex_count, f):
    vertices = [None] * vertex_count
    for i in range(vertex_count):
        v = tuple(map(float, f.readline().split()))
        assert len(v) == 3
        vertices[i] = v
    return vertices

def load_polygons():
    with open('teapot.off') as f:
        line = f.readline()
        assert line == 'OFF\n'

        vertex_count, face_count, edge_count = map(int, f.readline().split())

        vertices = load_vertices(vertex_count, f)

        polys = [None] * face_count
        for i in range(face_count):
            side_count, *face_vertices = map(int, f.readline().split())
            assert side_count == len(face_vertices)
            poly = list(map(vertices.__getitem__, face_vertices))
            polys[i] = poly

        assert f.read() == ''

    return polys

def triangulate(poly):
    if len(poly) < 3:
        raise ArgumentException("polygons must have at least 3 vertices")
    for i in range(1,len(poly) - 1):
        yield (poly[0], poly[i+1], poly[i])

def load_triangles(polys=None):
    tris = []
    if not polys:
        polys = load_polygons()
    for poly in polys:
        for tri in triangulate(poly):
            assert(len(tri)==3)
            for v in tri:
                assert(len(v)==3)
            tris.append(tri)
    return tris

In [ ]:
tris0 = load_triangles()

In [ ]:
# Find the bounding box of teacup.
min_x = min_y = min_z = float('inf')
max_x = max_y = max_z = -float('inf')
for tri in tris0:
    for vertex in tri:
        min_x = min(min_x, vertex[0])
        min_y = min(min_y, vertex[1])
        min_z = min(min_z, vertex[2])
        max_x = max(max_x, vertex[0])
        max_y = max(max_y, vertex[1])
        max_z = max(max_z, vertex[2])
print('Bounding box:')
print(f'x: [{min_x}, {max_x}]')
print(f'y: [{min_y}, {max_y}]')
print(f'z: [{min_z}, {max_z}]')

In [ ]:
draw_model(tris0)

Apply the transforms from the original `teapot.py`.

First, translate the center of the teapot to the origin.

In [ ]:
tris1 = transforms.polygon_map(transforms.translate_by((-0.5, 0.0, -0.6)), tris0)

In [ ]:
draw_model(tris1)

Then, rotate by x-axis so we see the side of the teapot, instead of the top.

In [ ]:
tris2 = transforms.polygon_map(transforms.rotate_x_by(-math.pi / 2), tris1)

In [ ]:
draw_model(tris2)

Finally, scale to make the teapot larger. Now we get the same image as the original `draw_teapot.py`, shown on the left side of Figure 4.2.

In [ ]:
tris = tris3 = transforms.polygon_map(transforms.scale_by(2), tris2)

In [ ]:
draw_model(tris)

**Note** The bottom of the teapot is missing or improperly drawn, as demonstrated by the following rotated views.

In [ ]:
draw_model(transforms.polygon_map(transforms.rotate_x_by(-math.pi / 4), tris))
draw_model(transforms.polygon_map(transforms.rotate_x_by(-math.pi / 2), tris))

**Figure 4.3** Use animation to show the individual images as the teapot is rotated by 45 degrees.

In [ ]:
draw_model(
    tris,
    get_matrix=lambda tick: transforms.rotate_z_by(min(tick // 100 % 60, 45) * math.pi / 180))

**Exercise 4.3** Scale the teapot by -1, and then rotate it to show all the images depicted in the exercise.

In [ ]:
draw_model(
    transforms.polygon_map(transforms.scale_by(-1), tris),
    get_matrix=lambda tick: transforms.rotate_x_by((tick // 2000 % 10) * math.pi / 5))

**Exercise 4.8** The statements given in the solution are not correct.
As demonstrated below, `compose(rotate_z_by(pi/2),rotate_x_by(pi/2))` is quite different from `rotate_y_by(pi/2)`.

In [ ]:
def compare_transforms(tr1, tr2, tris):
    tr1x = transforms.compose(transforms.translate_by((-1, 0, 0)), transforms.scale_by(0.5), tr1)
    tr2x = transforms.compose(transforms.translate_by(( 1, 0, 0)), transforms.scale_by(0.5), tr2)
    draw_model(transforms.polygon_map(tr1x, tris) + transforms.polygon_map(tr2x, tris))

In [ ]:
compare_transforms(
    transforms.compose(transforms.rotate_z_by(math.pi / 2), transforms.rotate_x_by(math.pi / 2)),
    transforms.rotate_y_by(math.pi / 2),
    tris)

Similarly, `compose(rotate_x_by(pi/2), rotate_z_by(pi/2))` is quite different from `rotate_y_by(-pi/2)`.

In [ ]:
compare_transforms(
    transforms.compose(transforms.rotate_x_by(math.pi / 2), transforms.rotate_z_by(math.pi / 2)),
    transforms.rotate_y_by(-math.pi / 2),
    tris)

On the other hand, `compose(rotate_z_by(pi/2), rotate_x_by(pi/2))`
is the same as `compose(rotate_x_by(pi/2), rotate_y_by(-pi/2))`.

In [ ]:
compare_transforms(
    transforms.compose(transforms.rotate_z_by(math.pi / 2), transforms.rotate_x_by(math.pi / 2)),
    transforms.compose(transforms.rotate_x_by(math.pi / 2), transforms.rotate_y_by(-math.pi / 2)),
    tris)

**Experiment: Rotate simple objects.**

In [ ]:
# An implementation of get_matrix function to keep rotating object along x-axis.
def x_rotator(period):
    initial_tick = -1
    def get_matrix(tick):
        # When pygame is initialized, tick is set to 0. However, by the time rendering starts,
        # a major fracion of a second has already passed. To ensure that we start rotation
        # from the initial position, we remember the time when get_matrix is called for the
        # first time.
        nonlocal initial_tick
        if initial_tick < 0:
            initial_tick = tick
        # There are 1000 ticks per second.
        angle = (tick - initial_tick) / 1000 / period * 2 * math.pi
        return transforms.rotate_x_by(angle)
    return get_matrix

In [ ]:
# A single triangle facing the viewer.
single = [[(*vectors.to_cartesian((1, angle * math.pi / 180)), 0.0) for angle in range(90, 90+360, 120)]]
print(single)

In [ ]:
# Show a rotating single triangle. Note that the triangle is invisible during half of the rotation.
draw_model(single, get_matrix=x_rotator(5))

In [ ]:
# Two triangles back to back.
double = single + transforms.polygon_map(transforms.rotate_y_by(math.pi), single)

In [ ]:
# Rotate both triangles together. They take turn to become visible.
draw_model(double, get_matrix=x_rotator(5))

**Experiment: Hidden surface elimination.**
* A triangle facing the viewer blocks the view of other triangles behind it.
* On the other hand, a triangle facing away from the user (and thus invisible) does not hide objects behind it.

In [ ]:
draw_model(
    single
    + transforms.polygon_map(transforms.compose(transforms.scale_by(0.7), transforms.rotate_x_by(    math.pi / 4)), single)  # Visible
    + transforms.polygon_map(transforms.compose(transforms.scale_by(0.7), transforms.rotate_x_by(3 * math.pi / 4)), single)) # Invisible